# Spatial Average comparison

This notebook will compare the following average methods.

- Generic
- cdutil (CDAT)
- xCDAT
- Xarray

## How to use this notebook
1. Install `xCDAT` and other packages `mamba install xcdat cdms2 cdutil pandas`
2. Run the notebook

In [1]:
import sys
import functools

import xcdat

import xarray as xr
import numpy as np
import pandas as pd

import cdms2
import cdutil
from genutil.averager import area_weights, __myGetAxisWeights
import MV2

# np.set_printoptions(suppress=True)

In [2]:
filename = 'gistemp1200_GHCNv4_ERSSTv5.nc'
var = 'tempanomaly'

In [3]:
rtol = 1e-7
atol = 1e-5

# relative error
relative_error = functools.partial(np.allclose, rtol=rtol, atol=0)
test_relative_error = functools.partial(np.testing.assert_allclose, rtol=rtol, atol=0)

# absolute error 
absolute_error = functools.partial(np.allclose, rtol=0, atol=atol)
test_absolute_error = functools.partial(np.testing.assert_allclose, rtol=0, atol=atol)

In [4]:
xcdat_ds = xcdat.open_dataset(filename)

In [5]:
cdat_ds = cdms2.open(filename)
cdat_da = cdat_ds(var)

# Method Comparison
For all methods we'll use weights generated by `xCDAT`, additionally we'll include a case of `CDAT` using it's own weights.

# Method break down
1. Generic
   - Uses [`np.sum`](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) and [`np.multiply`](https://numpy.org/doc/stable/reference/generated/numpy.multiply.html), [`np.sum`](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) is essential pairwise summation.
2. CDAT
   - Uses [`np.ma.average`](https://numpy.org/doc/stable/reference/generated/numpy.ma.average.html) which is just [`np.multiply`](https://numpy.org/doc/stable/reference/generated/numpy.multiply.html) and [`np.sum`](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) with mask handling
3. xCDAT
   - Uses [`xarray.DataArray.weighted`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.weighted.html) and [`xarray.core.weighted.DataArrayWeighted.mean`](https://docs.xarray.dev/en/stable/generated/xarray.core.weighted.DataArrayWeighted.mean.html#xarray.core.weighted.DataArrayWeighted.mean), see xArray below for further details.
4. Xarray
   - Uses [`np.einsum`](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html) for multiplication and summation. Subscript `...abc,...bc->a` is used for both.

# Takeaway
1. These results only inform us of relative differences between methods. Since floating point arithmetic has implied error via rounding and truncation, we can't assume just because the answers are different that one may be more/less accurate.
2. Verified that `xCDAT` produces same results as `Xarray`
4. Generic and `CDAT` differ, this could be due to masking between `np.ma` and replacing `nan` with zero
3. The generic method produces same results as `Xarray` when summation is done with `np.sum` 

In [6]:
averages = {}

xcdat_weights = xcdat_ds.spatial.get_weights(axis=['Y', 'X'], lat_bounds=None, lon_bounds=None)

cdat_weights = area_weights(cdat_da, axisoptions='yx')

In [7]:
def expand_weights(data, weights):
    weights = np.tile(np.expand_dims(weights, axis=0), (data.shape[0], 1, 1))
    
    mask = np.isnan(data)
    
    weights = np.where(~mask, weights, 0.)
    
    return weights, mask

def xarray_to_cdms2(data, var, ds, mask):
    masked_data = MV2.as_masked(data, mask=mask)
        
    transient_var = MV2.asVariable(masked_data)
    
    transient_var.setAxisList(ds[var].getAxisList())
    
    return transient_var
        
def weights_xarray_to_cdms2(var, ds, weights, output_ds):
    expanded, mask = expand_weights(ds[var], weights)
    
    converted = xarray_to_cdms2(expanded, var, output_ds, mask)
    
    return converted

def compare_items(data, func):
    comparison = [[func(x, y) for i, x in enumerate(data.values())] for j, y in enumerate(data.values())]
    
    names = data.keys()
    
    return pd.DataFrame(comparison, index=names, columns=names)

In [8]:
def generic_average(var, ds, weights=None):
    weights = np.tile(np.expand_dims(weights, axis=0), (ds.time.shape[0], 1, 1))
    
    weights = np.where(~np.isnan(ds[var]), weights, 0.)
    
    numerator = np.sum(weights * ds[var].copy(), axis=(1, 2))
    
    denominator = np.sum(weights, axis=(1, 2))
    
    return np.array(np.divide(numerator, denominator))

averages['generic'] = generic_average(var, xcdat_ds, weights=xcdat_weights)

In [9]:
def cdat_average(da, weights=None):
    return cdutil.averager(da.copy(), axis='yx', weights=weights).data
    
weights = xcdat_ds.spatial.get_weights(axis=['Y', 'X'], lat_bounds=None, lon_bounds=None)
    
averages['cdat'] = cdat_average(cdat_da, weights=cdat_weights)
averages['cdat_float32_weights'] = cdat_average(cdat_da, weights=cdat_weights.astype(np.float32))
averages['cdat_xcdat_weights'] = cdat_average(cdat_da, weights=weights_xarray_to_cdms2(var, xcdat_ds, weights, cdat_ds))

In [10]:
def xcdat_average(var, ds, weights=None):
    return ds.spatial.average(var, axis=['Y', 'X'], weights=weights)[var].data

averages['xcdat'] = xcdat_average(var, xcdat_ds, xcdat_weights)

In [11]:
def xarray_average(var, ds, weights=None, manual_sum=False):
    subscript = "...abc,...bc->...a" if not manual_sum else "...abc,...bc->...abc"
        
    weighted_sum = np.einsum(subscript, ds[var].fillna(0.).copy(), weights.copy())
    
    if manual_sum:
        weighted_sum = np.sum(weighted_sum, axis=(1, 2))
                
    mask = ds[var].notnull()
    
    sum_of_weights = np.einsum(subscript, mask, weights.copy())
    
    if manual_sum:
        sum_of_weights = np.sum(sum_of_weights, axis=(1, 2))
                
    sum_of_weights = np.where(sum_of_weights != 0., sum_of_weights, np.nan)
    
    return weighted_sum / sum_of_weights

averages["xarray"] = xarray_average(var, xcdat_ds, weights)
averages["xarray_manual_sum"] = xarray_average(var, xcdat_ds, weights, manual_sum=True)

### Relative error

In [12]:
compare_items(averages, relative_error)

,generic,cdat,cdat_float32_weights,cdat_xcdat_weights,xcdat,xarray,xarray_manual_sum
generic,True,False,False,False,False,False,True
cdat,False,True,False,False,False,False,False
cdat_float32_weights,False,False,True,False,False,False,False
cdat_xcdat_weights,False,False,False,True,False,False,False
xcdat,False,False,False,False,True,True,False
xarray,False,False,False,False,True,True,False
xarray_manual_sum,True,False,False,False,False,False,True


### Absolute error

In [13]:
compare_items(averages, absolute_error)

,generic,cdat,cdat_float32_weights,cdat_xcdat_weights,xcdat,xarray,xarray_manual_sum
generic,True,True,True,True,True,True,True
cdat,True,True,True,True,True,True,True
cdat_float32_weights,True,True,True,True,True,True,True
cdat_xcdat_weights,True,True,True,True,True,True,True
xcdat,True,True,True,True,True,True,True
xarray,True,True,True,True,True,True,True
xarray_manual_sum,True,True,True,True,True,True,True


In [14]:
compare_items(averages, lambda x, y: (np.max(np.abs(x - y)), np.min(np.abs(x - y)), np.round(np.count_nonzero(x != y)*100/xcdat_ds[var].shape[0], 2)))

,generic,cdat,cdat_float32_weights,cdat_xcdat_weights,xcdat,xarray,xarray_manual_sum
generic,"(0.0, 0.0, 0.0)","(1.7580544797723974e-07, 8.484379865336678e-12...","(2.2649765e-06, 0.0, 96.07)","(9.536743e-07, 0.0, 91.79)","(9.536743e-06, 0.0, 98.83)","(9.536743e-06, 0.0, 98.83)","(0.0, 0.0, 0.0)"
cdat,"(1.7580544797723974e-07, 8.484379865336678e-12...","(0.0, 0.0, 0.0)","(2.2855772432439636e-06, 1.3306960741643614e-1...","(9.345794523829554e-07, 7.929020981456425e-11,...","(9.540234808369519e-06, 2.3563563253392594e-10...","(9.540234808369519e-06, 2.3563563253392594e-10...","(1.7580544797723974e-07, 8.484379865336678e-12..."
cdat_float32_weights,"(2.2649765e-06, 0.0, 96.07)","(2.2855772432439636e-06, 1.3306960741643614e-1...","(0.0, 0.0, 0.0)","(2.026558e-06, 0.0, 96.83)","(8.821487e-06, 0.0, 98.77)","(8.821487e-06, 0.0, 98.77)","(2.2649765e-06, 0.0, 96.07)"
cdat_xcdat_weights,"(9.536743e-07, 0.0, 91.79)","(9.345794523829554e-07, 7.929020981456425e-11,...","(2.026558e-06, 0.0, 96.83)","(0.0, 0.0, 0.0)","(9.298325e-06, 0.0, 99.18)","(9.298325e-06, 0.0, 99.18)","(9.536743e-07, 0.0, 91.79)"
xcdat,"(9.536743e-06, 0.0, 98.83)","(9.540234808369519e-06, 2.3563563253392594e-10...","(8.821487e-06, 0.0, 98.77)","(9.298325e-06, 0.0, 99.18)","(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(9.536743e-06, 0.0, 98.83)"
xarray,"(9.536743e-06, 0.0, 98.83)","(9.540234808369519e-06, 2.3563563253392594e-10...","(8.821487e-06, 0.0, 98.77)","(9.298325e-06, 0.0, 99.18)","(0.0, 0.0, 0.0)","(0.0, 0.0, 0.0)","(9.536743e-06, 0.0, 98.83)"
xarray_manual_sum,"(0.0, 0.0, 0.0)","(1.7580544797723974e-07, 8.484379865336678e-12...","(2.2649765e-06, 0.0, 96.07)","(9.536743e-07, 0.0, 91.79)","(9.536743e-06, 0.0, 98.83)","(9.536743e-06, 0.0, 98.83)","(0.0, 0.0, 0.0)"


# xCDAT vs CDAT (cdutil)

xCDAT uses Xarrays [weighted](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.weighted.html) and [mean](https://docs.xarray.dev/en/stable/generated/xarray.core.weighted.DataArrayWeighted.mean.html#xarray.core.weighted.DataArrayWeighted.mean) methods.

Sources:

- https://github.com/xCDAT/xcdat/blob/007fb8a95bdd0904351cc9dc89f3ace027e8a520/xcdat/spatial.py#L732-L741
- https://github.com/pydata/xarray/blob/0496cb441a237c5edc0727e77b1be83e1dde6c67/xarray/core/weighted.py#L273-L285

CDAT's (cdutil/genutil) averager uses [np.ma.average](https://numpy.org/doc/stable/reference/generated/numpy.ma.average.html)

Notes:

- CDAT averager transpose's to (lat, lon, time) and reduces using `np.ma.average`
- CDAT lat weights == 0.5 * xCDAT lat weights
- CDAT lon weights == xCDAT lat weights / 360.

Sources:

- https://github.com/CDAT/genutil/blob/59517ab54e65c03098502f63434270f96020f3eb/Lib/averager.py#L1283-L1308
- https://github.com/CDAT/genutil/blob/59517ab54e65c03098502f63434270f96020f3eb/Lib/averager.py#L845-L852
- https://github.com/CDAT/cdms/blob/3f8c7baa359f428628a666652ecf361764dc7b7a/Lib/MV2.py#L505-L516

Items to investigate:

- How does np.ma differ from Xarrays masking?
- When using xCDAT weights, why does CDAT's results still differ? Does above have an correlation?

# Takeaway
- CDAT's answers still differ even when using the same weights as xCDAT
  - Could be related to differences np.ma vs. Xarray masking
- CDAT generates weights differently than xCDAT
  - CDAT lat weights == 0.5 * xCDAT lat weights
  - CDAT lon weights == xCDAT lon weights / 360.

In [15]:
# Summed up version of CDAT average
# Validates same results as CDAT output
def test_raw_cdat_average(da, weights, to_compare):
    da_ma = np.ma.array(da.copy()).transpose([1, 2, 0])
    
    weights_ma = np.ma.array(weights.copy()).transpose([1, 2, 0])
    
    for _ in range(2):
        da_ma, weights_ma = np.ma.average(da_ma, weights=weights_ma, axis=0, returned=1)
    
    test_relative_error(da_ma.data, to_compare)
    test_absolute_error(da_ma.data, to_compare)
    
test_raw_cdat_average(cdat_da, cdat_weights, averages['cdat'])

In [16]:
# xCDAT weights are different than CDAT weights
np.allclose(xcdat_weights.data, cdat_weights[0].data)

False

In [17]:
# CDAT average with xCDAT weights results in 98.3% different values than xCDAT output
test_raw_cdat_average(cdat_da, weights_xarray_to_cdms2(var, xcdat_ds, weights, cdat_ds), averages['xcdat'])

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 1677 / 1706 (98.3%)
Max absolute difference: 9.298325e-06
Max relative difference: 0.00012282
 x: array([-0.180887, -0.26498 , -0.09773 , ...,  0.856695,  0.906705,
        0.894914], dtype=float32)
 y: array([-0.180887, -0.26498 , -0.09773 , ...,  0.856693,  0.906703,
        0.894913], dtype=float32)

## Breakdown of CDAT weight generation

In [18]:
# area_weights function
cdat_lat_bnds = cdat_da.getAxis(1).getBounds()
cdat_lon_bnds = cdat_da.getAxis(2).getBounds()

# getWeights function
# cdat_lat_wts, cdat_lon_wts = cdat_da.getGrid().getWeights()
cdat_lat_wts = 0.5 * np.absolute(np.sin((np.pi / 180.0) * cdat_lat_bnds[:, 1]) - np.sin((np.pi / 180.0) * cdat_lat_bnds[:, 0]))
cdat_lon_wts = np.absolute((cdat_lon_bnds[:, 1] - cdat_lon_bnds[:, 0])) / 360.0
# getWeights end

# area_weights function continued
wt = np.outer(cdat_lat_wts, cdat_lon_wts)

# __myGetAxisWeights function
# newaxiswt = __myGetAxisWeights(cdat_da, 0, 'weighted')
axis_bounds = cdat_da.getAxis(0).getBounds()

axis_wts = np.abs(axis_bounds[..., 1] - axis_bounds[...,0])

newaxiswt = axis_wts
# __myGetAxisWeights end

# area_weights function continued
wtlist = list(wt.shape)
wtlist.insert(0, newaxiswt.shape[0])

new_wtshape = tuple(wtlist)
wt = np.resize(wt, new_wtshape)

new_newaxiswt_shape = list(newaxiswt.shape)

for nn in range(1, len(wt.shape), 1):
    new_newaxiswt_shape.append(1)
    
newaxiswt = np.resize(newaxiswt, tuple(new_newaxiswt_shape))

wt = wt * newaxiswt

np.allclose(wt, cdat_weights)

True

## Breakdown of xCDAT weight generation

In [19]:
from functools import reduce

lat_bnds = xcdat_ds.bounds.get_bounds("Y")
# np.abs(lat_bnds[: 1] - lat_bnds[: 0]) same as CDAT
lat_weights = xcdat_ds.spatial._get_latitude_weights(lat_bnds, None)

lon_bnds = xcdat_ds.bounds.get_bounds("X")
lon_weights = xcdat_ds.spatial._get_longitude_weights(lon_bnds, None)

xcdat_wts = reduce((lambda x, y: x * y), [lat_weights, lon_weights])

np.allclose(xcdat_wts, xcdat_weights.data)

True

In [20]:
# Bounds are the same
np.allclose(cdat_lat_bnds.data, lat_bnds.data), np.allclose(cdat_lon_bnds.data, lon_bnds.data)

(True, True)

In [21]:
# Weights don't match
np.allclose(cdat_lat_wts.data, lat_weights.data), np.allclose(cdat_lon_wts, lon_weights.data)

(False, False)

In [22]:
# Lat weight's off by 0.5 and lon weight's off by 0.002777778 (1/360.0)
np.allclose(cdat_lat_wts.data, (lat_weights * 0.5).data), np.allclose(cdat_lon_wts, (lon_weights / 360.0).data)

(True, True)

# xCDAT vs Generic

xCDAT uses Xarrays [weighted](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.weighted.html) and [mean](https://docs.xarray.dev/en/stable/generated/xarray.core.weighted.DataArrayWeighted.mean.html#xarray.core.weighted.DataArrayWeighted.mean) methods. Drilling down Xarray's methods use [np.einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html) to perform calculations.

`np.einsum` subscript `...abc,...bc->a` broadcasts and multiplies `bc` of each array summing along the `a` axis. So `np.einsum('...abc,...bc->...a', D, W)` is equal to `np.sum(np.multiply(D, W), axis=(1, 2))`.

Sources:

- https://github.com/xCDAT/xcdat/blob/007fb8a95bdd0904351cc9dc89f3ace027e8a520/xcdat/spatial.py#L732-L741
- https://github.com/pydata/xarray/blob/0496cb441a237c5edc0727e77b1be83e1dde6c67/xarray/core/weighted.py#L273-L285

Generic uses [np.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) and [np.divide](https://numpy.org/doc/stable/reference/generated/numpy.divide.html) methods.

## Notes

- `np.sum` is `np.add` reduced over `axis`
- `np.sum` will use a `partial pairwise summation`(https://numpy.org/doc/stable/reference/generated/numpy.sum.html)
- `np.einsum` will use [`blas`](https://en.wikipedia.org/wiki/OpenBLAS) or [`lapack`](https://en.wikipedia.org/wiki/LAPACK)
- `math.fsum` uses slower, more precise algorithm than `np.sum`

## Takeaway

- xCDAT differs from generic
  - Could be related to masking differences
- xCDAT similar when using `np.sum` outside of `np.einsum`, indicates that underlying libraries may handle round-off error differently
- `math.fsum` may be more precise than `np.sum`

In [23]:
# Show which underlying libraries numpy is using
np.show_config()

blas_info:
    libraries = ['cblas', 'blas', 'cblas', 'blas']
    library_dirs = ['/opt/conda/lib']
    include_dirs = ['/opt/conda/include']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    define_macros = [('NO_ATLAS_INFO', 1), ('HAVE_CBLAS', None)]
    libraries = ['cblas', 'blas', 'cblas', 'blas']
    library_dirs = ['/opt/conda/lib']
    include_dirs = ['/opt/conda/include']
    language = c
lapack_info:
    libraries = ['lapack', 'blas', 'lapack', 'blas']
    library_dirs = ['/opt/conda/lib']
    language = f77
lapack_opt_info:
    libraries = ['lapack', 'blas', 'lapack', 'blas', 'cblas', 'blas', 'cblas', 'blas']
    library_dirs = ['/opt/conda/lib']
    language = c
    define_macros = [('NO_ATLAS_INFO', 1), ('HAVE_CBLAS', None)]
    include_dirs = ['/opt/conda/include']
Supported SIMD extensions in this NumPy install:
    baseline = SSE,SSE2,SSE3
    found = SSSE3,SSE41,POPCNT,SSE42,AVX,F16C,FMA3,AVX2
    not found = AVX512F,AVX512CD,AVX512_KNL,AVX

In [24]:
!mamba list | grep -E "blas|lapack"

libblas                   3.9.0           16_linux64_openblas    conda-forge
libcblas                  3.9.0           16_linux64_openblas    conda-forge
liblapack                 3.9.0           16_linux64_openblas    conda-forge
libopenblas               0.3.21          pthreads_h78a6416_2    conda-forge
openblas                  0.3.21          pthreads_h320a7e8_2    conda-forge


### Quick look at calculating numerator

In [25]:
data_no_nan = xcdat_ds[var].fillna(0.).data
weights_no_nan = xcdat_weights.fillna(0.).data

In [26]:
generic_output = np.sum(np.multiply(data_no_nan, weights_no_nan), axis=(1, 2))

In [27]:
# Interesting to look at fsum, numpy mentions it being more accurate
import math

generic_fsum_output = np.multiply(data_no_nan, weights_no_nan)
generic_fsum_output = [math.fsum(x.flatten()) for x in generic_fsum_output]
generic_fsum_output = np.array(generic_fsum_output)

relative_error(generic_output, generic_fsum_output), absolute_error(generic_output, generic_fsum_output)

(False, False)

In [28]:
einsum_output = np.einsum('...abc,...bc->...a', data_no_nan, weights_no_nan)

relative_error(generic_output, einsum_output), absolute_error(generic_output, einsum_output)

(False, False)

In [29]:
einsum_alt_output = np.sum(np.einsum('...abc,...bc->...abc', data_no_nan, weights_no_nan), axis=(1, 2))

relative_error(generic_output, einsum_alt_output), absolute_error(generic_output, einsum_alt_output)

(True, True)